In [6]:
!pip uninstall -y transformers accelerate


Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0


In [7]:
!pip install transformers==4.38.2 accelerate==0.27.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.0 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.


In [1]:
import torch
import torch.nn as nn
import os
from google.colab import drive

# Mount Drive
drive.mount('/content/drive', force_remount=True)

# --- CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FEATURE_DIR = "/content/drive/MyDrive/FYP_Full_Project/features"


Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel

class VideoGPT2Captioner(nn.Module):
    def __init__(self, visual_dim=2304, gpt2_dim=768, prefix_len=5):
        super().__init__()

        self.prefix_len = prefix_len
        self.gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

        # Visual → GPT2 space (multi-token)
        self.projection = nn.Sequential(
            nn.Linear(visual_dim, gpt2_dim * prefix_len),
            nn.LayerNorm(gpt2_dim * prefix_len)
        )

        # Learned start token (VERY important)
        self.start_token = nn.Parameter(torch.randn(1, 1, gpt2_dim))

    def encode_visual(self, visual_features):
        """
        visual_features: [B, 2304]
        returns: [B, prefix_len+1, 768]
        """
        B = visual_features.size(0)

        proj = self.projection(visual_features)
        prefix = proj.view(B, self.prefix_len, 768)

        start = self.start_token.expand(B, -1, -1)
        return torch.cat([start, prefix], dim=1)


In [3]:
import pandas as pd
TEST_CSV_PATH = "/content/drive/MyDrive/FYP_Full_Project/splits/test.csv"
df = pd.read_csv(TEST_CSV_PATH)
print(df.columns)

Index(['path', 'caption', 'gif_id'], dtype='object')


In [7]:
GIF_DIR = "/content/drive/MyDrive/FYP_Full_Project/gifs"
os.makedirs(GIF_DIR, exist_ok=True)


In [8]:
import os
import requests
from tqdm import tqdm

def download_gif(url, save_path):
    try:
        r = requests.get(url, timeout=20)
        if r.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(r.content)
            return True
    except Exception as e:
        return False

for idx, row in tqdm(df.iterrows(), total=len(df)):
    gif_id = row["gif_id"]
    url = row["path"]

    save_path = os.path.join(GIF_DIR, f"{gif_id}.gif")

    if not os.path.exists(save_path):
        download_gif(url, save_path)


100%|██████████| 1000/1000 [03:30<00:00,  4.76it/s]


In [25]:
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image

class GifCaptionDataset(Dataset):
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        gif_id = self.df.iloc[idx]["gif_id"]
        caption = self.df.iloc[idx]["caption"]

        gif_path = os.path.join(GIF_DIR, f"{gif_id}.gif")
        image = Image.open(gif_path).convert("RGB")

        return {
            "image": image,      # raw PIL
            "caption": caption,
            "gif_id": gif_id
        }



In [10]:
import torchvision.transforms as T

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

TEST_CSV_PATH = "/content/drive/MyDrive/FYP_Full_Project/splits/test.csv"

test_ds = GifCaptionDataset(
    csv_file=TEST_CSV_PATH,
    gif_dir=GIF_DIR,
    transform=transform
)



In [11]:
sample = test_ds[0]

print(sample["gif_id"])
print(sample["image"].shape)
print(sample["caption"])


gif_4343
torch.Size([3, 224, 224])
a man is making symbols with his hands and dancing


In [22]:

from transformers import VideoMAEModel, ViTModel, AutoImageProcessor

# vit = ViTModel.from_pretrained("google/vit-base-patch16-224")
# vit.to(DEVICE)
# vit.eval()

# --- A. Load Extractors ---
print("Loading Feature Extractors...")
action_proc = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
action_model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base").to(DEVICE).eval()

vit_proc = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224").to(DEVICE).eval()



Loading Feature Extractors...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/377M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from PIL import Image, ImageSequence
import torch

def load_gif_frames(gif_path, num_frames=16):
    gif = Image.open(gif_path)

    frames = []
    for frame in ImageSequence.Iterator(gif):
        frames.append(frame.convert("RGB"))

    # Uniform sampling
    if len(frames) >= num_frames:
        idx = torch.linspace(0, len(frames) - 1, num_frames).long()
        frames = [frames[i] for i in idx]
    else:
        # Pad if too short
        frames = frames + [frames[-1]] * (num_frames - len(frames))

    return frames


In [14]:
@torch.no_grad()
def extract_vit_feature(image):
    inputs = vit_proc(images=image, return_tensors="pt").to(DEVICE)
    outputs = vit_model(**inputs)
    cls_feat = outputs.last_hidden_state[:, 0, :]  # [1, 768]
    return cls_feat.squeeze(0)


In [15]:
@torch.no_grad()
def extract_action_feature(frames):
    inputs = action_proc(
        images=frames,
        return_tensors="pt"
    ).to(DEVICE)

    outputs = action_model(**inputs)

    # Temporal average pooling
    feat = outputs.last_hidden_state.mean(dim=1)  # [1, 768]
    return feat.squeeze(0)


In [16]:
def extract_emotion_feature(image):
    return extract_vit_feature(image)


In [23]:
def extract_visual_features(sample):
    image = sample["image"]
    gif_id = sample["gif_id"]

    gif_path = os.path.join(GIF_DIR, f"{gif_id}.gif")

    frames = load_gif_frames(gif_path)

    f_app = extract_vit_feature(image)
    f_act = extract_action_feature(frames)
    f_emo = extract_emotion_feature(image)

    visual_feat = torch.cat([f_app, f_act, f_emo], dim=-1)  # [2304]

    return visual_feat


In [4]:
from transformers import GPT2Tokenizer

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = VideoGPT2Captioner(
    visual_dim=2304,
    prefix_len=5
).to(DEVICE)

checkpoint_path = "/content/drive/MyDrive/FYP_Full_Project/model_final_v5.pth"
state = torch.load(checkpoint_path, map_location=DEVICE)

# This is OK even if the old model didn't have start_token
model.load_state_dict(state, strict=False)

model.eval()
print("✅ Model loaded successfully")


✅ Model loaded successfully


In [19]:
@torch.no_grad()
def generate_caption(model, visual_feat):
    """
    visual_feat: [2304] tensor
    """

    model.eval()

    visual_feat = visual_feat.unsqueeze(0).to(DEVICE)

    # Encode visual prefix
    prefix_embeds = model.encode_visual(visual_feat)

    prefix_len = prefix_embeds.size(1)
    attention_mask = torch.ones((1, prefix_len), device=DEVICE)

    output_ids = model.gpt2.generate(
        inputs_embeds=prefix_embeds,
        attention_mask=attention_mask,

        max_new_tokens=20,

        # 🔒 ALIGNMENT SETTINGS (IMPORTANT)
        do_sample=False,       # ❌ no sampling
        num_beams=3,           # ✅ beam search
        repetition_penalty=1.8,

        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [24]:
print("🎬 FINAL MODEL OUTPUTS\n")

for i in [0, 50, 100]:
    sample = test_ds[i]

    visual_feat = extract_visual_features(sample)
    pred = generate_caption(model, visual_feat)

    print(f"Sample {i} | GIF: {sample['gif_id']}")
    print("GT :", sample["caption"])
    print("PR :", pred)
    print("-" * 60)


🎬 FINAL MODEL OUTPUTS



ValueError: The image to be converted to a PIL image contains values outside the range [0, 1], got [-1.878157377243042, 2.640000104904175] which cannot be converted to uint8.

In [ ]:
print("🎬 FINAL MODEL OUTPUTS\n")

for i in [0, 50, 100]:
    sample = test_ds[i]

    image = sample["image"]
    gt_caption = sample["caption"]

    visual_feat = extract_visual_features(image)

    pred = generate_caption(model, visual_feat)

    print(f"Sample {i}")
    print("GT :", gt_caption)
    print("PR :", pred)
    print("-" * 50)



In [28]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image, ImageSequence
from transformers import AutoImageProcessor, ViTModel, VideoMAEModel, GPT2Tokenizer

# -------------------
# 1️⃣ Setup
# -------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GIF_DIR = "/content/drive/MyDrive/FYP_Full_Project/gifs"
TEST_CSV_PATH = "/content/drive/MyDrive/FYP_Full_Project/splits/test.csv"
CHECKPOINT_PATH = "/content/drive/MyDrive/FYP_Full_Project/model_final_v5.pth"

# -------------------
# 2️⃣ Load Feature Extractors
# -------------------
print("Loading Feature Extractors...")

# Action/Video features
action_proc = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
action_model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base").to(DEVICE).eval()

# Appearance features
vit_proc = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224").to(DEVICE).eval()

# -------------------
# 3️⃣ Load Caption Model & Tokenizer
# -------------------
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = VideoGPT2Captioner(
    visual_dim=2304,
    prefix_len=5
).to(DEVICE)

state = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
model.load_state_dict(state, strict=False)
model.eval()
print("✅ Caption model loaded successfully")

# -------------------
# 4️⃣ Dataset
# -------------------
import pandas as pd

class GifCaptionDataset(Dataset):
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        gif_id = self.df.iloc[idx]["gif_id"]
        caption = self.df.iloc[idx]["caption"]
        gif_path = os.path.join(GIF_DIR, f"{gif_id}.gif")
        image = Image.open(gif_path).convert("RGB")
        return {
            "image": image,     # raw PIL image
            "caption": caption,
            "gif_id": gif_id
        }

test_ds = GifCaptionDataset(TEST_CSV_PATH)

# -------------------
# 5️⃣ Helper Functions
# -------------------
# Load multiple frames from a GIF
def load_gif_frames(gif_path, num_frames=16):
    gif = Image.open(gif_path)
    frames = [frame.convert("RGB") for frame in ImageSequence.Iterator(gif)]

    # Uniform sampling / padding
    if len(frames) >= num_frames:
        idx = torch.linspace(0, len(frames) - 1, num_frames).long()
        frames = [frames[i] for i in idx]
    else:
        frames = frames + [frames[-1]] * (num_frames - len(frames))

    return frames

# Extract ViT feature (appearance)
@torch.no_grad()
def extract_vit_feature(image):
    inputs = vit_proc(images=image, return_tensors="pt").to(DEVICE)
    outputs = vit_model(**inputs)
    cls_feat = outputs.last_hidden_state[:, 0, :]
    return cls_feat.squeeze(0)

# Extract VideoMAE feature (action)
@torch.no_grad()
def extract_action_feature(frames):
    inputs = action_proc(images=frames, return_tensors="pt").to(DEVICE)
    outputs = action_model(**inputs)
    feat = outputs.last_hidden_state.mean(dim=1)
    return feat.squeeze(0)

# Extract emotion feature (placeholder, optional)
def extract_emotion_feature(image):
    return extract_vit_feature(image)

# Fuse features to 2304-dim
def extract_visual_features(sample):
    image = sample["image"]
    gif_id = sample["gif_id"]
    gif_path = os.path.join(GIF_DIR, f"{gif_id}.gif")

    frames = load_gif_frames(gif_path)

    f_app = extract_vit_feature(image)
    f_act = extract_action_feature(frames)
    f_emo = extract_emotion_feature(image)

    visual_feat = torch.cat([f_app, f_act, f_emo], dim=-1)  # [2304]
    return visual_feat

# -------------------
# 6️⃣ Caption Generation
# -------------------
@torch.no_grad()
def generate_caption(model, visual_feat, max_len=20):
    visual_feat = visual_feat.unsqueeze(0).to(DEVICE)
    prefix_embeds = model.encode_visual(visual_feat)
    prefix_len = prefix_embeds.size(1)
    attention_mask = torch.ones((1, prefix_len), device=DEVICE)

    output_ids = model.gpt2.generate(
        inputs_embeds=prefix_embeds,
        attention_mask=attention_mask,
        max_new_tokens=max_len,
        do_sample=False,
        num_beams=3,
        repetition_penalty=1.8,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# -------------------
# 7️⃣ Run Inference
# -------------------
print("🎬 Running Inference...\n")

for i in [0, 50, 100]:  # sample indices
    sample = test_ds[i]
    visual_feat = extract_visual_features(sample)
    pred = generate_caption(model, visual_feat)

    print(f"Sample {i} | GIF: {sample['gif_id']}")
    print("GT :", sample["caption"])
    print("PR :", pred)
    print("-" * 60)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


Loading Feature Extractors...


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Caption model loaded successfully
🎬 Running Inference...



/usr/local/lib/python3.12/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  return torch.tensor(value)


Sample 0 | GIF: gif_4343
GT : a man is making symbols with his hands and dancing
PR : a man in a white shirt and black hat is smiling.
------------------------------------------------------------
Sample 50 | GIF: gif_5044
GT : two men in front of microphones are nodding at each other.
PR : a man is talking to a woman and she smiles at him.
------------------------------------------------------------
Sample 100 | GIF: gif_248
GT : woman with in black outfit poses and blinks.
PR : a man is talking to a woman and she is smiling.
------------------------------------------------------------


In [29]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image, ImageSequence
from transformers import (
    AutoImageProcessor,
    ViTModel,
    VideoMAEModel,
    GPT2Tokenizer,
    AutoFeatureExtractor,
    AutoModel
)

# -------------------
# 1️⃣ Setup
# -------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GIF_DIR = "/content/drive/MyDrive/FYP_Full_Project/gifs"
TEST_CSV_PATH = "/content/drive/MyDrive/FYP_Full_Project/splits/test.csv"
CHECKPOINT_PATH = "/content/drive/MyDrive/FYP_Full_Project/model_final_v5.pth"

# -------------------
# 2️⃣ Load Feature Extractors
# -------------------
print("Loading Feature Extractors...")

# Appearance features (ViT)
vit_proc = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224").to(DEVICE).eval()

# Action features (VideoMAE)
action_proc = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
action_model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base").to(DEVICE).eval()

# Emotion features (example: FER+ pre-trained)
emotion_proc = AutoFeatureExtractor.from_pretrained("nateraw/ferplus")  # placeholder
emotion_model = AutoModel.from_pretrained("nateraw/ferplus").to(DEVICE).eval()

# -------------------
# 3️⃣ Load Caption Model & Tokenizer
# -------------------
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = VideoGPT2Captioner(
    visual_dim=2304 + 128,  # 2304 + emotion embedding dim
    prefix_len=5
).to(DEVICE)

state = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
model.load_state_dict(state, strict=False)
model.eval()
print("✅ Caption model loaded successfully")

# -------------------
# 4️⃣ Dataset
# -------------------
import pandas as pd

class GifCaptionDataset(Dataset):
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        gif_id = self.df.iloc[idx]["gif_id"]
        caption = self.df.iloc[idx]["caption"]
        gif_path = os.path.join(GIF_DIR, f"{gif_id}.gif")
        image = Image.open(gif_path).convert("RGB")
        return {
            "image": image,
            "caption": caption,
            "gif_id": gif_id
        }

test_ds = GifCaptionDataset(TEST_CSV_PATH)

# -------------------
# 5️⃣ Helper Functions
# -------------------

def load_gif_frames(gif_path, num_frames=16):
    gif = Image.open(gif_path)
    frames = [frame.convert("RGB") for frame in ImageSequence.Iterator(gif)]

    # Uniform sampling / padding
    if len(frames) >= num_frames:
        idx = torch.linspace(0, len(frames) - 1, num_frames).long()
        frames = [frames[i] for i in idx]
    else:
        frames = frames + [frames[-1]] * (num_frames - len(frames))

    return frames

# Appearance feature
@torch.no_grad()
def extract_vit_feature(image):
    inputs = vit_proc(images=image, return_tensors="pt").to(DEVICE)
    outputs = vit_model(**inputs)
    cls_feat = outputs.last_hidden_state[:, 0, :]
    return cls_feat.squeeze(0)

# Action feature
@torch.no_grad()
def extract_action_feature(frames):
    inputs = action_proc(images=frames, return_tensors="pt").to(DEVICE)
    outputs = action_model(**inputs)
    feat = outputs.last_hidden_state.mean(dim=1)
    return feat.squeeze(0)

# Emotion feature
@torch.no_grad()
def extract_emotion_feature(image):
    inputs = emotion_proc(images=image, return_tensors="pt").to(DEVICE)
    outputs = emotion_model(**inputs)
    feat = outputs.last_hidden_state.mean(dim=1)
    return feat.squeeze(0)

# Fuse all features
def extract_visual_features(sample):
    image = sample["image"]
    gif_id = sample["gif_id"]
    gif_path = os.path.join(GIF_DIR, f"{gif_id}.gif")

    frames = load_gif_frames(gif_path)

    f_app = extract_vit_feature(image)
    f_act = extract_action_feature(frames)
    f_emo = extract_emotion_feature(image)  # real emotion embedding

    visual_feat = torch.cat([f_app, f_act, f_emo], dim=-1)
    return visual_feat

# -------------------
# 6️⃣ Caption Generation
# -------------------
@torch.no_grad()
def generate_caption(model, visual_feat, max_len=20):
    visual_feat = visual_feat.unsqueeze(0).to(DEVICE)
    prefix_embeds = model.encode_visual(visual_feat)
    prefix_len = prefix_embeds.size(1)
    attention_mask = torch.ones((1, prefix_len), device=DEVICE)

    output_ids = model.gpt2.generate(
        inputs_embeds=prefix_embeds,
        attention_mask=attention_mask,
        max_new_tokens=max_len,
        do_sample=False,
        num_beams=5,  # more beams -> better predictions
        repetition_penalty=1.8,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# -------------------
# 7️⃣ Run Inference
# -------------------
print("🎬 Running Emotion-aware Inference...\n")

for i in [0, 50, 100]:
    sample = test_ds[i]
    visual_feat = extract_visual_features(sample)
    pred = generate_caption(model, visual_feat)

    print(f"Sample {i} | GIF: {sample['gif_id']}")
    print("GT :", sample["caption"])
    print("PR :", pred)
    print("-" * 60)


Loading Feature Extractors...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


OSError: nateraw/ferplus is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [9]:
!pip install transformers opencv-python facenet-pytorch moviepy deepface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.0 MB/s eta 0:00:00


In [11]:
import torch
from transformers import AutoImageProcessor, ViTModel, VideoMAEModel, GPT2Tokenizer
import numpy as np
from PIL import Image, ImageSequence
import pandas as pd
import os
from deepface import DeepFace

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------- Feature Extractors -------------------
vit_proc = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224").to(DEVICE).eval()

action_proc = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
action_model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base").to(DEVICE).eval()

# ------------------- Dataset -------------------
class GifCaptionDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, gif_dir):
        self.df = pd.read_csv(csv_file)
        self.gif_dir = gif_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        gif_id = row["gif_id"]
        caption = row["caption"]
        gif_path = os.path.join(self.gif_dir, f"{gif_id}.gif")
        image = Image.open(gif_path).convert("RGB")
        return {"image": image, "caption": caption, "gif_id": gif_id}

# ------------------- Utilities -------------------
def load_gif_frames(gif_path, num_frames=16):
    gif = Image.open(gif_path)
    frames = [frame.convert("RGB") for frame in ImageSequence.Iterator(gif)]
    if len(frames) >= num_frames:
        idx = torch.linspace(0, len(frames)-1, num_frames).long()
        frames = [frames[i] for i in idx]
    else:
        frames += [frames[-1]] * (num_frames - len(frames))
    return frames

@torch.no_grad()
def extract_vit_feature(image):
    inputs = vit_proc(images=image, return_tensors="pt").to(DEVICE)
    outputs = vit_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze(0)

@torch.no_grad()
def extract_action_feature(frames):
    inputs = action_proc(images=frames, return_tensors="pt").to(DEVICE)
    outputs = action_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze(0)

def extract_emotion_feature(image):
    img_np = np.array(image)
    try:
        # Use DeepFace to predict emotions
        analysis = DeepFace.analyze(img_np, actions=['emotion'], enforce_detection=False)
        emotions = analysis['emotion']
        # Convert dict to tensor
        prob = torch.tensor(list(emotions.values()), dtype=torch.float32)
    except Exception:
        prob = torch.zeros(7)
        prob[0] = 1.0  # fallback neutral
    return prob.to(DEVICE)

def extract_visual_features(sample):
    image = sample["image"]
    gif_path = f"/content/drive/MyDrive/FYP_Full_Project/gifs/{sample['gif_id']}.gif"
    frames = load_gif_frames(gif_path)
    f_app = extract_vit_feature(image)
    f_act = extract_action_feature(frames)
    f_emo = extract_emotion_feature(image)
    return torch.cat([f_app, f_act, f_emo], dim=-1)

# ------------------- GPT2 Caption -------------------
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = VideoGPT2Captioner(visual_dim=2304+7, prefix_len=5).to(DEVICE)
model.load_state_dict(torch.load("/content/drive/MyDrive/FYP_Full_Project/model_final_v5.pth", map_location=DEVICE), strict=False)
model.eval()

@torch.no_grad()
def generate_caption(model, visual_feat, max_len=20):
    visual_feat = visual_feat.unsqueeze(0).to(DEVICE)
    prefix_embeds = model.encode_visual(visual_feat)
    attention_mask = torch.ones((1, prefix_embeds.size(1)), device=DEVICE)

    output_ids = model.gpt2.generate(
        inputs_embeds=prefix_embeds,
        attention_mask=attention_mask,
        max_new_tokens=max_len,
        do_sample=False,
        num_beams=5,
        repetition_penalty=1.8,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# ------------------- Run Inference -------------------
test_ds = GifCaptionDataset("/content/drive/MyDrive/FYP_Full_Project/splits/test.csv",
                            "/content/drive/MyDrive/FYP_Full_Project/gifs")

for i in [0, 50, 100]:
    sample = test_ds[i]
    visual_feat = extract_visual_features(sample)
    pred = generate_caption(model, visual_feat)
    print(f"Sample {i} | GIF: {sample['gif_id']}")
    print("GT :", sample["caption"])
    print("PR :", pred)
    print("-"*50)


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5


26-01-19 04:33:23 - 🔗 facial_expression_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5 to /root/.deepface/weights/facial_expression_model_weights.h5...


100%|██████████| 5.98M/5.98M [00:00<00:00, 72.4MB/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1543 and 2311x3840)

In [22]:
# ----------------------------
# 1️⃣ Imports & Device Setup
# ----------------------------
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import ViTModel, ViTFeatureExtractor
from transformers import VideoMAEModel, VideoMAEFeatureExtractor
from fer import FER
import cv2
import numpy as np
from PIL import Image

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# 2️⃣ Load GPT2 tokenizer & trained model
# ----------------------------
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# This class must match what you trained
class VideoGPT2Captioner(nn.Module):
    def __init__(self, visual_dim=2304, prefix_len=5):
        super().__init__()
        self.gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
        self.visual_proj = nn.Linear(visual_dim, self.gpt2.config.n_embd)
        self.prefix_len = prefix_len

    def forward(self, visual_feat, input_ids=None):
        # Project visual features to GPT2 embeddings
        prefix_emb = self.visual_proj(visual_feat).unsqueeze(1)  # (B, 1, n_embd)
        batch_size = prefix_emb.shape[0]

        # Create dummy input for GPT2 if none provided
        if input_ids is None:
            input_ids = torch.full(
                (batch_size, 1), tokenizer.pad_token_id, dtype=torch.long
            ).to(DEVICE)

        # Embed input_ids
        inputs_embeds = self.gpt2.transformer.wte(input_ids)
        # Concatenate visual prefix
        inputs_embeds = torch.cat([prefix_emb, inputs_embeds], dim=1)
        outputs = self.gpt2(inputs_embeds=inputs_embeds)
        return outputs

# Load trained model
model = VideoGPT2Captioner(visual_dim=2304, prefix_len=5).to(DEVICE)
checkpoint_path = "/content/drive/MyDrive/FYP_Full_Project/model_final_v5.pth"
state = torch.load(checkpoint_path, map_location=DEVICE)
model.load_state_dict(state, strict=False)
model.eval()
print("✅ GPT2 trained captioner loaded successfully")

# ----------------------------
# 3️⃣ Load Visual Feature Extractors
# ----------------------------
print("Loading ViT (frame-level visual features)...")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224").to(DEVICE)
vit_model.eval()
vit_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

print("Loading VideoMAE (temporal motion features)...")
videomae_model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base").to(DEVICE)
videomae_model.eval()
videomae_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base")

# ----------------------------
# 4️⃣ Load Emotion Detector
# ----------------------------
print("Loading FER (emotion features)...")
fer_detector = FER(mtcnn=False)  # works with numpy images

# ----------------------------
# 5️⃣ Helper Functions
# ----------------------------

def extract_visual_features(sample_frames):
    """
    Input: list of frames (H x W x C)
    Output: concatenated ViT + VideoMAE features
    """
    # ViT features (frame-level)
    vit_feats = []
    for frame in sample_frames:
        frame_pil = Image.fromarray(frame)
        inputs = vit_extractor(images=frame_pil, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            feat = vit_model(**inputs).last_hidden_state[:, 0, :]  # CLS token
        vit_feats.append(feat)
    vit_feats = torch.stack(vit_feats).mean(0)  # average over frames

    # VideoMAE features (temporal)
    video_input = videomae_extractor(
        sample_frames, return_tensors="pt", do_normalize=True
    ).pixel_values.to(DEVICE)  # shape: (B, C, T, H, W)
    with torch.no_grad():
        vid_feat = videomae_model(video_input).last_hidden_state.mean(1)  # mean pooling
    vid_feat = vid_feat.squeeze(0)

    # Concatenate ViT + VideoMAE features
    visual_feat = torch.cat([vit_feats, vid_feat], dim=-1)
    return visual_feat

def extract_emotion_features(frame):
    """
    Returns a vector of emotion probabilities
    """
    emotions = fer_detector.detect_emotions(frame)
    if emotions:
        return torch.tensor(list(emotions[0]["emotions"].values()), dtype=torch.float32).to(DEVICE)
    else:
        return torch.zeros(7).to(DEVICE)  # 7 emotion classes

def fuse_visual_emotion(visual_feat, emotion_feat):
    """
    Simple concatenation fusion of visual and emotion features
    """
    return torch.cat([visual_feat, emotion_feat], dim=-1)

def generate_caption(model, fused_feat, max_len=30):
    """
    Generate a caption from fused visual+emotion features
    """
    model.eval()
    input_ids = torch.full((1, 1), tokenizer.pad_token_id, dtype=torch.long).to(DEVICE)
    generated = input_ids
    for _ in range(max_len):
        outputs = model(fused_feat.unsqueeze(0), generated)
        next_token = outputs.logits[:, -1, :].argmax(-1)
        generated = torch.cat([generated, next_token.unsqueeze(0)], dim=1)
        if next_token.item() == tokenizer.eos_token_id:
            break
    caption = tokenizer.decode(generated.squeeze().tolist(), skip_special_tokens=True)
    return caption

# ----------------------------
# 6️⃣ Example Usage
# ----------------------------
# Sample GIF/Video frames as numpy arrays (H x W x C)
# For example, frames = [frame0, frame1, frame2, ...]
# Here you can extract frames using cv2.VideoCapture for GIF/MP4

frames = []  # TODO: load your video frames here as numpy arrays
if len(frames) == 0:
    print("⚠️ No frames loaded! Please provide frames from a video/GIF.")

# Example with first frame for emotion
if len(frames) > 0:
    emotion_feat = extract_emotion_features(frames[0])
    visual_feat = extract_visual_features(frames)
    fused_feat = fuse_visual_emotion(visual_feat, emotion_feat)
    caption = generate_caption(model, fused_feat)
    print("🎬 Generated caption (emotion-aware):", caption)


✅ GPT2 trained captioner loaded successfully
Loading ViT (frame-level visual features)...


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading VideoMAE (temporal motion features)...
Loading FER (emotion features)...
⚠️ No frames loaded! Please provide frames from a video/GIF.


In [28]:
# ==========================================
# Emotion-aware GIF/Video Captioning Pipeline
# ==========================================

# 1️⃣ Install dependencies (run once)
!pip install torch torchvision transformers opencv-python pillow facenet-pytorch fer

# 2️⃣ Imports
import torch
import torch.nn as nn
import torchvision.transforms as T
from transformers import ViTModel, VideoMAEModel, GPT2Tokenizer
from fer import FER
from PIL import Image, ImageSequence
import numpy as np
import cv2

# 3️⃣ Device & tokenizer
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# 4️⃣ Load feature models
print("Loading ViT...")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224").to(DEVICE)
vit_model.eval()

print("Loading VideoMAE...")
videomae_model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base").to(DEVICE)
videomae_model.eval()

print("Loading FER (emotion)...")
fer_detector = FER(mtcnn=True)
print("✅ Feature models loaded\n")


INPUT_DIM = 2304 + 768 + 128  # appearance + action + emotion dims
VOCAB_SIZE = len(tokenizer)

model = VideoGPT2Captioner(visual_dim=INPUT_DIM, prefix_len=5).to(DEVICE)
checkpoint_path = "/content/drive/MyDrive/FYP_Full_Project/model_final_v5.pth"
state = torch.load(checkpoint_path, map_location=DEVICE)
model.load_state_dict(state, strict=False)
model.eval()
print("✅ Trained GPT2 Captioner loaded\n")

# 6️⃣ Helper functions

@torch.no_grad()
def extract_appearance_feature(image):
    transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])
    img_tensor = transform(image).unsqueeze(0).to(DEVICE)
    feat = vit_model(img_tensor).last_hidden_state.mean(dim=1)  # 1 x 768
    return feat

@torch.no_grad()
def extract_action_feature(frames):
    transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])
    frame_tensor = torch.stack([transform(f) for f in frames]).unsqueeze(0).to(DEVICE)
    feat = videomae_model(frame_tensor).last_hidden_state.mean(dim=1)  # 1 x 768
    return feat

@torch.no_grad()
def extract_emotion_feature(image):
    np_img = np.array(image)
    results = fer_detector.detect_emotions(np_img)
    if len(results) == 0:
        return torch.zeros(128).to(DEVICE)
    emo_probs = list(results[0]["emotions"].values())
    emo_feat = torch.tensor(emo_probs, dtype=torch.float32).to(DEVICE)
    if len(emo_feat) < 128:
        emo_feat = torch.cat([emo_feat, torch.zeros(128-len(emo_feat)).to(DEVICE)])
    return emo_feat.unsqueeze(0)

@torch.no_grad()
def generate_caption(model, visual_feat, max_len=50):
    tokens = model.generate(visual_feat, max_length=max_len, tokenizer=tokenizer)
    caption = tokenizer.decode(tokens[0], skip_special_tokens=True)
    return caption

def gif_to_frames(gif_path, num_frames=8):
    gif = Image.open(gif_path)
    frames = []
    total_frames = getattr(gif, "n_frames", 1)  # get total frames safely
    step = max(1, total_frames // num_frames)

    for i in range(total_frames):
        gif.seek(i)
        if i % step == 0:
            frames.append(gif.convert("RGB"))
        if len(frames) == num_frames:
            break
    return frames

# 7️⃣ Run inference on a GIF/video
gif_path = "/content/tumblr_l876j3kjpF1qcw5xjo1_250.gif"  # Replace with your GIF/video path
frames = gif_to_frames(gif_path, num_frames=8)
image = frames[0]  # first frame for emotion + appearance

# Extract features
f_app = extract_appearance_feature(image)
f_act = extract_action_feature(frames)
f_emo = extract_emotion_feature(image)

# Concatenate all features
visual_feat = torch.cat([f_app, f_act, f_emo], dim=-1)

# Generate caption
caption = generate_caption(model, visual_feat)
print("🎬 Emotion-aware Caption:", caption)


Loading ViT...


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading VideoMAE...
Loading FER (emotion)...
✅ Feature models loaded

✅ Trained GPT2 Captioner loaded



RuntimeError: The size of tensor a (784) must match the size of tensor b (1568) at non-singleton dimension 1

In [35]:
import torch
from transformers import AutoImageProcessor, ViTModel, GPT2Tokenizer
from transformers import VideoMAEFeatureExtractor, VideoMAEModel
from PIL import Image, ImageSequence
import pandas as pd
from deepface import DeepFace
from torchvision import transforms
import numpy as np
import requests
from io import BytesIO

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------------
# 1️⃣ Load Models
# ------------------------------
print("Loading ViT (appearance)...")
vit_proc = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224").to(DEVICE).eval()

print("Loading VideoMAE (action)...")
action_proc = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base")
action_model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base").to(DEVICE).eval()

print("Loading FER (emotion)...")
def extract_emotion_feature(image):
    result = DeepFace.analyze(np.array(image), actions=['emotion'], enforce_detection=False)
    emo_vec = [result['emotion'][k] for k in sorted(result['emotion'].keys())]
    return torch.tensor(emo_vec, dtype=torch.float32).unsqueeze(0).to(DEVICE)

print("Loading trained GPT2 captioner...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

VOCAB_SIZE = tokenizer.vocab_size
INPUT_DIM = 2304  # adjust to your model's input dim

model = VideoGPT2Captioner(visual_dim=INPUT_DIM, prefix_len=5).to(DEVICE)
checkpoint_path = "/content/drive/MyDrive/FYP_Full_Project/model_final_v5.pth"
state = torch.load(checkpoint_path, map_location=DEVICE)
model.load_state_dict(state, strict=False)
model.eval()

print("✅ All models loaded successfully.\n")

# ------------------------------
# 2️⃣ Helper Functions
# ------------------------------
def fetch_gif(url):
    response = requests.get(url)
    response.raise_for_status()
    return Image.open(BytesIO(response.content))

def gif_to_frames(gif, num_frames=8):
    frames = []
    total = getattr(gif, "n_frames", 1)
    step = max(1, total // num_frames)
    for i, frame in enumerate(ImageSequence.Iterator(gif)):
        if i % step == 0:
            frames.append(frame.convert("RGB"))
        if len(frames) == num_frames:
            break
    return frames

def extract_appearance_feature(image):
    inputs = vit_proc(images=image, return_tensors="pt").to(DEVICE)
    outputs = vit_model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # CLS token

video_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def extract_action_feature(frames):
    processed = [video_transform(f) for f in frames]
    video_tensor = torch.stack(processed).unsqueeze(0).to(DEVICE)  # batch x frames x 3 x H x W
    outputs = action_model(video_tensor)
    return outputs.last_hidden_state.mean(dim=1)  # pooled feature

def generate_caption(model, visual_feat):
    input_ids = torch.tensor([tokenizer.pad_token_id]).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        outputs = model.generate(visual_feat, input_ids=input_ids, max_length=50)
    caption = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return caption

# ------------------------------
# 3️⃣ Load Test CSV
# ------------------------------
test_csv = "/content/drive/MyDrive/FYP_Full_Project/splits/test.csv"
df_test = pd.read_csv(test_csv)

# ------------------------------
# 4️⃣ Run Inference
# ------------------------------
print("🎬 Running emotion-aware captions...\n")
for idx, row in df_test.iterrows():
    gif_url = row['path']  # column with URLs
    try:
        gif = fetch_gif(gif_url)
        frames = gif_to_frames(gif, num_frames=8)
        image = frames[0]  # first frame for appearance + emotion

        # Extract features
        f_app = extract_appearance_feature(image)
        f_act = extract_action_feature(frames)
        f_emo = extract_emotion_feature(image)

        # Concatenate features for GPT2 input
        visual_feat = torch.cat([f_app, f_act, f_emo], dim=1)

        # Generate caption
        caption = generate_caption(model, visual_feat)
        print(f"GIF URL: {gif_url}\nCaption: {caption}\n{'-'*60}")

    except Exception as e:
        print(f"Error processing {gif_url}: {e}")


Loading ViT (appearance)...


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading VideoMAE (action)...
Loading FER (emotion)...
Loading trained GPT2 captioner...
✅ All models loaded successfully.

🎬 Running emotion-aware captions...

Error processing https://38.media.tumblr.com/tumblr_m5bsogjFSi1qzc18no1_400.gif: The size of tensor a (784) must match the size of tensor b (1568) at non-singleton dimension 1
Error processing https://31.media.tumblr.com/1de4b2cfe3c39f0b8923eaa883f0de71/tumblr_nodj8jxg791qkqcdko1_400.gif: The size of tensor a (784) must match the size of tensor b (1568) at non-singleton dimension 1
Error processing https://38.media.tumblr.com/2b7c307a664fd23700534deaaffda905/tumblr_npy2w5VDCU1qdj2rpo1_400.gif: The size of tensor a (784) must match the size of tensor b (1568) at non-singleton dimension 1
Error processing https://38.media.tumblr.com/24f7c4fbfca82ab9a6774fa944af428b/tumblr_nkq1w3A69N1scqhj0o1_400.gif: The size of tensor a (784) must match the size of tensor b (1568) at non-singleton dimension 1
Error processing https://38.media.tum